## Noisy CRC

> By Utaha
> 
> I just learned that CRC can leak information about the original text, so I added noises to make it secure even if you can choose the generator polynomial! Good luck!
> 
> `nc chals.sekai.team 3005`
> 
> Attachment: chall.py

We can choose a CRC16 polynomial and the server gives us the CRC of a secret number jumbled with 2 other random 16 bit integers. We aren't allowed to send the same polynomial twice which prevents us from selecting numbers which appear twice. Identifying the secret number lets us decyrpt the flag.

Mathematically, CRC behaves like polynomials in $F_2$, where the CRC of some ciphertext is the remainder of the ciphertext polynomial divided by the CRC polynomial. The oracle we are given is equivalent to $Q(f(x)) = s(x) \bmod f(x)$ for some fixed secret $s(x)$[^1] and user-supplied modulus $f(x)$ where $s(x), f(x) \isin \mathcal{P}_n(F_2)$.

To find which are the true CRCs among the noise, we want some way to tell if two different CRCs using different moduli came from the same source input. We can use a property of the modulo operation and think about using **composite moduli**. If we fix a small "integrity modulus" polynomial $g(x)$ and vary a large polynomial $h_i(x)$ such that $f_i(x) = g(x) \cdot h_i(x)$ has degree 16, two true CRCs taken residues modulo $g(x)$ should be equal:

$$
\begin{align*}
Q(g(x) \cdot h_i(x)) &= s(x) \bmod g(x) \cdot h_i(x) \\
Q(g(x) \cdot h_i(x)) \bmod g(x) &= \Bigl( s(x) \bmod g(x) \cdot h_i(x) \Bigr) \bmod g(x) \\
&= s(x) \bmod g(x)
\end{align*}
$$

By querying with multiple CRC polynomials of this form, find the most common remainder when each of the CRCs are taken modulo $g(x)$. Any CRC which does not have this remainder must have been from random noise. After filtering, select the instances where only one candidate is left and this must be a true CRC. The secret can then be recovered using the chinese remainder theorem. We know we have the correct secret if the resulting polynomial from CRT only has coefficients up to $x^{512+16}$. The flag can then be decrypted.

[^1]: Technically it is $s(x) \cdot x^{16}$ because CRC16 will shift the input data 16 bits left before processing.

In [1]:
from pwn import *
from tqdm.auto import tqdm

Fn = GF(2)
Rn.<x> = PolynomialRing(Fn)

r = remote("chals.sekai.team", int(3005))
r.recvuntil(b"flag: ")
flag_enc = r.recvline().strip().decode()

def oracle_send(x): # helper to query with polynomial
    x = ZZ(list(x.change_ring(ZZ)), 2)
    r.sendline(f"{x}".encode())

def oracle_recv(): # helper to return polynomial
    r.recvuntil(b"ial: ")
    crcs = safeeval.expr(r.recvline())
    crcs = [ Rn(ZZ(crc).bits()) for crc in crcs ]
    return crcs

# Query 100 sets of g(x) * h_i(x)
n = 100
integrity_modulus = x^4 + x + 1
assert integrity_modulus.is_irreducible()

queries = []
while len(queries) < n:
    query = Rn.random_element(degree=12)
    # In case random_element collides
    if query not in queries:
        queries.append(query)
        oracle_send(query * integrity_modulus)

results = []
for _ in tqdm(range(n)):
    results.append(oracle_recv())

[x] Opening connection to chals.sekai.team on port 3005
[x] Opening connection to chals.sekai.team on port 3005: Trying 34.148.151.228
[+] Opening connection to chals.sekai.team on port 3005: Done


  0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
from collections import Counter
from Crypto.Cipher import AES
from hashlib import sha256
from Crypto.Util.number import long_to_bytes

# Find the most common remainder when taken modulo integrity_modulus
integrity_check_ctr = Counter([
    j % integrity_modulus
    for i in results
    for j in i  
])
top_three_integrity_remainders = integrity_check_ctr.most_common(3)
print("top (s(x) % g(x), count):", top_three_integrity_remainders)
integrity_remainder = top_three_integrity_remainders[0][0]
print("using integrity remainder", integrity_remainder)

# Most common residue must have appeared at least n times
assert top_three_integrity_remainders[0][1] >= n 
assert top_three_integrity_remainders[1][1] < n

# Recover CRT parts
crt_res, crt_mod = [], []
for query, result in zip(queries, results):
    remainders_passing_integrity_check = [
        result_i for result_i in result
        if result_i % integrity_modulus == integrity_remainder
    ]
    if len(remainders_passing_integrity_check) == 1:
        crt_res.append(remainders_passing_integrity_check[0])
        crt_mod.append(query)
print(f"Found {len(crt_mod)} remainder / moduli pairs passing integrity check")

secret = CRT_list(crt_res, crt_mod)
print(secret)
secret = ZZ(list(secret.change_ring(ZZ)), 2) // 2^16
print(secret)

cipher = AES.new(
    sha256(long_to_bytes(secret)).digest()[:16],
    AES.MODE_CTR,
    nonce=b"12345678"
)
enc_flag = cipher.decrypt(bytes.fromhex(flag_enc))
print(enc_flag)

top (s(x) % g(x), count): [(x^3 + x^2, 113), (x^3 + x, 18), (x, 15)]
using integrity remainder x^3 + x^2
Found 87 remainder / moduli pairs passing integrity check
x^527 + x^524 + x^522 + x^521 + x^520 + x^519 + x^518 + x^517 + x^516 + x^515 + x^513 + x^511 + x^509 + x^508 + x^506 + x^505 + x^501 + x^500 + x^498 + x^496 + x^495 + x^494 + x^491 + x^490 + x^489 + x^487 + x^485 + x^484 + x^482 + x^481 + x^480 + x^478 + x^477 + x^476 + x^466 + x^465 + x^464 + x^459 + x^458 + x^457 + x^456 + x^455 + x^454 + x^449 + x^448 + x^446 + x^443 + x^441 + x^439 + x^435 + x^434 + x^433 + x^427 + x^426 + x^425 + x^424 + x^421 + x^413 + x^410 + x^408 + x^407 + x^406 + x^405 + x^404 + x^401 + x^400 + x^399 + x^396 + x^394 + x^393 + x^392 + x^390 + x^388 + x^386 + x^384 + x^381 + x^380 + x^376 + x^373 + x^370 + x^369 + x^368 + x^367 + x^363 + x^361 + x^360 + x^359 + x^357 + x^353 + x^349 + x^346 + x^343 + x^338 + x^332 + x^331 + x^330 + x^328 + x^327 + x^325 + x^324 + x^320 + x^319 + x^318 + x^316 + x^315